In [1]:
import pandas as pd
import json
import os
import sys

import itertools

import pandas as pd
sys.path.insert(1, '/scratch/cinthiasouza/mv-text-summarizer')


In [2]:
%load_ext autoreload
%autoreload 2

from src import preprocess
from src import tokenizer

In [3]:
def get_id_sections (data, sections, key_name='section_names'):

    section_id = {}
    for key, values in sections.items():
        controle=False
        aux = []
        for name in values:
            for i in range(len(data[key_name])):
                if data[key_name][i].find(name) != -1:
                    aux.append(i)
        try:
            section_id[key] = min(aux)
        except ValueError:
            pass
    
    return section_id

In [4]:
def get_section_by_id(data, section_id):
    
    article = {'id': data['article_id'], 'abstract': ' '.join(data['abstract_text'])}

    for key, values in section_id.items():

        if key == 'introduction':
            try:
                aux = data['sections'][values:section_id['materials']]
                merged  = list(itertools.chain(*aux))
                article[key] = merged
            except KeyError:
                article[key] = data['sections'][values]
        elif key == 'materials':
            try:
                aux = data['sections'][values:section_id['conclusion']]
                merged  = list(itertools.chain(*aux))
                article[key] = merged
            except KeyError:
                article[key] = data['sections'][values]
        elif key == 'conclusion':
            
                sections_names = data['section_names']
                if data['section_names'][-1].find('acknowledgment') != -1:
                    sections_names = data['section_names'][:len(data['section_names'])-1]
                    
                aux = data['sections'][values: len(sections_names)]
                merged  = list(itertools.chain(*aux))
                article[key] = merged
                
    return article


In [5]:
def get_sections_plosone(article, section_id):
    
    data = {'journal_title': article['journal_title'],
            'id': article['id'],
            'doi': article['doi'],
            'title': article['title'],
            'keywords': article['keywords'],
            'abstract': article['abstract'],
            'title_sections': article['title_sections']}

    for key, item in section_id.items():

        if key == 'introduction':
            aux = []
            try:
                for i in range(item,  section_id.get('materials')):
                    aux.append(article[article['title_sections'][i]])
                
                text = str(" ".join(aux))
                text = preprocess.format_xml(text)
                text = preprocess.xml_to_text(text)
                text = preprocess.format_intro(text)
                text = (text.encode('ascii', 'ignore')).decode("utf-8")

                text = tokenizer.split_sentences([text])
                text = list(map(str, text[0]))

                data[key] = text
                
                
            except TypeError:
                try:
                    for i in range(item,  section_id.get('conclusion')):
                        aux.append(article[article['title_sections'][i]])
                    
                    text = str(" ".join(aux))
                    text = preprocess.format_xml(text)
                    text = preprocess.xml_to_text(text)
                    text = (text.encode('ascii', 'ignore')).decode("utf-8")
            
                    text = tokenizer.split_sentences([text])
                    text = list(map(str, text[0]))
            
                    data[key] = text
                
                except TypeError:
                    for i in range(item,  len(article['title_sections'])):
                        if (article['title_sections'][i] != 'acknowledgements') and (article['title_sections'][i] != 'references'):
                            aux.append(article[article['title_sections'][i]])
                    
                    text = str(" ".join(aux))
                    text = preprocess.format_xml(text)
                    text = preprocess.xml_to_text(text)
                    text = (text.encode('ascii', 'ignore')).decode("utf-8")
            
                    text = tokenizer.split_sentences([text])
                    text = list(map(str, text[0]))
            
                    data[key] = text
                    
        elif key == 'materials':
            aux = []
            try:
                for i in range(item,  section_id.get('conclusion')):
                    aux.append(article[article['title_sections'][i]])
            except TypeError:
                for i in range(item,  len(article['title_sections'])):
                    if (article['title_sections'][i] != 'acknowledgements') and (article['title_sections'][i] != 'references'):
                        aux.append(article[article['title_sections'][i]])

            text = str(" ".join(aux))
            text = preprocess.format_xml(text)
            text = preprocess.xml_to_text(text)
            text = (text.encode('ascii', 'ignore')).decode("utf-8")
            
            text = tokenizer.split_sentences([text])
            text = list(map(str, text[0]))
            
            data[key] = text
            
        elif key == 'conclusion':
            aux = []
            for i in range(item,  len(article['title_sections'])):
                if (article['title_sections'][i] != 'acknowledgements') and (article['title_sections'][i] != 'references'):
                    aux.append(article[article['title_sections'][i]])

            text = str(" ".join(aux))
            text = preprocess.format_xml(text)
            text = preprocess.xml_to_text(text)
            text = preprocess.remove_unicode(text)
            text = (text.encode('ascii', 'ignore')).decode("utf-8")
            
            text = tokenizer.split_sentences([text])
            text = list(map(str, text[0]))
            
            data[key] = text
            
    return data

In [6]:
def main_segmentation(path_to_read, path_to_write, dataset='plosone'):
    
    articles = []
    
    if (dataset == 'arxiv') or (dataset == 'pubmed'):
        with open(path_to_read, 'r') as f:
            data = f.readlines()
    elif dataset == 'plosone':
        data = os.listdir(path_to_read)


    sections = {'introduction':['introduction'],
    'materials': ['subject', 'experiment', 'method', 'material', 'procedur'],
    'conclusion': ['analysis', 'discuss', 'result', 'conclusion', 'experiments', 'review']}

    batch_files = []
    count = 0

    for i in range(len(data)):

        if (dataset == "arxiv") or (dataset == 'pubmed'):
            
            article = json.loads(data[i])
            sections_id = get_id_sections (article, sections, key_name='section_names')
            article_segmented = get_section_by_id(article, sections_id)
            
            batch_files.append(article_segmented)
                                    
        elif dataset == "plosone":
            try:
                with open("{}/{}".format(path_to_read, data[i]), 'r') as f:
                    article = json.load(f)
                
                sections_id = get_id_sections (article, sections, key_name='title_sections')
                article_segmented = get_sections_plosone(article, sections_id)
                
                batch_files.append(article_segmented)
            except:
                
                pass

        if (count % 10000 == 0) or (count == len(data)-1):

            print("Número de arquivos salvos: {}".format(count))

            with open("{}/batch_{}.txt".format(path_to_write, count), 'w', encoding='utf-8') as f:
                for text in batch_files:
                    aux = json.dumps(text)
                    f.write(aux)
                    f.write("\n")

            batch_files = []

        count+=1

In [7]:
paths = ['PMC001xxxxxx_pp', 'PMC002xxxxxx_pp', 'PMC003xxxxxx_pp', 'PMC004xxxxxx_pp', 'PMC005xxxxxx_pp', 'PMC006xxxxxx_pp', 'PMC007xxxxxx_pp']

In [8]:
path_to_read = "../../../plosone/PMC001xxxxxx_pp/"
path_to_write = '../../../plosone/test'

In [16]:
path_to_read = "../../../pubmed-dataset/val.txt"
path_to_write = '../../../pubmed-dataset/valid'

In [14]:
path_to_read = "../../../arxiv-dataset/val.txt"
path_to_write = '../../../arxiv-dataset/valid'

In [17]:
main_segmentation(path_to_read, path_to_write, dataset='pubmed')

Número de arquivos salvos: 0
Número de arquivos salvos: 6632


In [9]:
path_base='../../../plosone/train'

In [10]:
with open('{}/{}'.format(path_base, 'batch_0.txt', 'r')) as f:
    files = f.readlines()

In [11]:
data = json.loads(files[0])

In [12]:
data['introduction'][0]

'introductionThere is much current interest in investigating the ability of redox-inactive metal ions to modulate redox reactions by virtue of their Lewis acidity, particularly with respect to their possible roles in O2 evolution1 and activation.2,3 For example, the oxygen-evolving complex of Photosystem II requires a redox-inactive Ca2+ ion to produce O2.1 In addition, redox-inactive ions have been found to affect the stability and reactivities of high-valent metal-oxo complexes in biomimetic systems2 as well as to accelerate O2 activation by FeII and MnII complexes.3'